In [5]:
from textblob import TextBlob
import sys
import tweepy
from twython import Twython
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import socket
import ssl
import nltk
nltk.download('all')


from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Error loading all: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [6]:
# Read in config file from local
config = pd.read_csv('/Users/tim/Desktop/Projects/config/config_2.csv')

In [7]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [8]:
# Authentication
auth = tweepy.auth.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
twitterApi = tweepy.API(auth, wait_on_rate_limit = True)

try:
  twitterApi.verify_credentials()
  print('Authenticated')
except:
  print('Error authenticating')

Authenticated


In [9]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)
keyword = input('Please enter twitter account: ')
noOfTweet = int(input ('Please enter how many tweets to analyze: '))
tweets = tweepy.Cursor(twitterApi.search_tweets, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
 
 #print(tweet.text)
 tweet_list.append(tweet.text)
 analysis = TextBlob(tweet.text)
 score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 polarity += analysis.sentiment.polarity
 
if neg > pos:
    negative_list.append(tweet.text)
    negative += 1
elif pos > neg:
    positive_list.append(tweet.text)
    positive += 1
elif pos == neg:
    neutral_list.append(tweet.text)
    neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')
print(score)

Please enter twitter account: SpaceX
Please enter how many tweets to analyze: 100
{'neg': 0.0, 'neu': 0.867, 'pos': 0.133, 'compound': 0.1531}


In [10]:
twitterAccount = keyword

In [11]:
tweets = tweepy.Cursor(twitterApi.user_timeline, 
                        screen_name=keyword, 
                        count=None,
                        since_id=None,
                        max_id=None,
                        trim_user=True,
                        exclude_replies=True,
                        contributor_details=False,
                        include_entities=False
                        ).items(noOfTweet)

In [12]:
df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweet'])

Unexpected parameter: contributor_details
Unexpected parameter: include_entities
Unexpected parameter: contributor_details
Unexpected parameter: include_entities
Unexpected parameter: contributor_details
Unexpected parameter: include_entities
Unexpected parameter: contributor_details
Unexpected parameter: include_entities
Unexpected parameter: contributor_details
Unexpected parameter: include_entities
Unexpected parameter: contributor_details
Unexpected parameter: include_entities


In [13]:
df.head()

,Tweet
0,More photos from last night's Falcon 9 launch ...
1,"Deployment confirmed, @NASA’s DART is on its w..."
2,Second stage engine restart and escape burn co...
3,Falcon 9’s first stage booster landed on the O...
4,"Second stage engine cutoff confirmed, now in c..."


In [14]:
# Cleaning the tweets

def cleanUpTweet(txt):
    # Remove mentions
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
    return txt

In [15]:
df['Tweet'] = df['Tweet'].apply(cleanUpTweet)

In [18]:
def getTextSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

def getTextPolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [19]:
df['Subjectivity'] = df['Tweet'].apply(getTextSubjectivity)
df['Polarity'] = df['Tweet'].apply(getTextPolarity)

In [20]:
df.head(noOfTweet)

,Tweet,Subjectivity,Polarity
0,More photos from last night's Falcon 9 launch ...,0.188889,0.166667
1,"Deployment confirmed, ’s DART is on its way to...",1.000000,0.400000
2,Second stage engine restart and escape burn co...,0.200000,0.050000
3,Falcon 9’s first stage booster landed on the O...,0.466667,0.375000
4,"Second stage engine cutoff confirmed, now in c...",0.500000,0.200000
...,...,...,...
95,LIVE NOW: A Dragon spacecraft returns to Eart...,0.500000,0.318182
96,"After its ~one-month stay at the , the Dragon ...",0.250000,0.250000
97,"On Thurs., Sept. 30, that could help us bette...",0.500000,0.500000
98,More pics from return →,0.500000,0.500000


In [21]:
df = df.drop(df[df['Tweet'] == ''].index)

In [22]:
df.head(noOfTweet)

,Tweet,Subjectivity,Polarity
0,More photos from last night's Falcon 9 launch ...,0.188889,0.166667
1,"Deployment confirmed, ’s DART is on its way to...",1.000000,0.400000
2,Second stage engine restart and escape burn co...,0.200000,0.050000
3,Falcon 9’s first stage booster landed on the O...,0.466667,0.375000
4,"Second stage engine cutoff confirmed, now in c...",0.500000,0.200000
...,...,...,...
95,LIVE NOW: A Dragon spacecraft returns to Eart...,0.500000,0.318182
96,"After its ~one-month stay at the , the Dragon ...",0.250000,0.250000
97,"On Thurs., Sept. 30, that could help us bette...",0.500000,0.500000
98,More pics from return →,0.500000,0.500000


In [23]:
# negative, nautral, positive analysis
def getTextAnalysis(a):
    if a < 0:
        return "Negative"
    elif a == 0:
        return "Neutral"
    else:
        return "Positive"

In [24]:
df['Score'] = df['Polarity'].apply(getTextAnalysis)